#### The Travel Concierge
- Here's an agent that helps you book a trip.
- The agent's logic is:
  - Look at the message history.
  - Check which "slots" (Destination, Date, Class) are still missing.
  - If something is missing, ask for it.
  - If everything is present, "book" the trip.



In [1]:
from typing import Annotated, Optional
from typing_extensions import TypedDict
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

from dotenv import load_dotenv   # from the python-dotenv library - load the OpenAI API key from file
load_dotenv(dotenv_path='../../../.env', override=True)

True

In [2]:
from typing import Annotated, Optional
from typing_extensions import TypedDict
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

# 1. We add specific fields to our State
class State(TypedDict):
    messages: Annotated[list, add_messages]
    # These fields stay None until the agent finds them in conversation
    destination: Optional[str]
    travel_date: Optional[str]
    travel_class: Optional[str]

class BookingInfo(BaseModel):
    destination: Optional[str] = Field(None, description="The city the user wants to visit")
    travel_date: Optional[str] = Field(None, description="The day or date of travel")
    travel_class: Optional[str] = Field(None, description="Economy, Business, or First class")
    assistant_message: str = Field(..., description="The message to speak back to the user")

llm = ChatOpenAI(model="gpt-4o").with_structured_output(BookingInfo)

def concierge_node(state: State):
    # Instructions to the LLM to act as a 'Slot Filler'
    system_prompt = (
        "You are a travel assistant. Current Data:\n"
        f"Destination: {state.get('destination')}\n"
        f"Date: {state.get('travel_date')}\n"
        f"Class: {state.get('travel_class')}\n"
        "If data is missing, ask the user. If you just received data, "
        "reply with 'I have noted [data].' and ask for the next item."
    )
    
    messages = [("system", system_prompt)] + state["messages"]
    response = llm.invoke(messages)
    
    # --- HERE IS THE KEY: LOGIC TO UPDATE THE STATE FIELDS ---
    # In a real app, you'd use 'Tool Calling' for this. 
    # For this simple example, we'll just check if the user's last message 
    # contained certain keywords to update the state.
    
    return {
        "messages": [("assistant", response.assistant_message)],
        "destination": response.destination or state.get("destination"),
        "travel_date": response.travel_date or state.get("travel_date"),
        "travel_class": response.travel_class or state.get("travel_class")
    }

   

# 2. Build the Graph
builder = StateGraph(State)
builder.add_node("concierge", concierge_node)
builder.add_edge(START, "concierge")
builder.add_edge("concierge", END)

memory = MemorySaver()
app = builder.compile(checkpointer=memory)

# 3. THE CLI LOOP (Now it checks the slots!)
config = {"configurable": {"thread_id": "user_789"}}

print("Welcome! I need your destination, date, and class.")

while True:
    u_input = input("User: ")
    if u_input.lower() in ["quit", "exit"]: break

    # Run the agent
    result = app.invoke({"messages": [("user", u_input)]}, config)
    
    print(f"Assistant: {result['messages'][-1].content}")

    # NOW THE CODE CHECKS THE STATE
    is_destination_filled = result.get("destination")
    is_date_filled = result.get("travel_date")
    is_class_filled = result.get("travel_class")

    if is_destination_filled and is_date_filled and is_class_filled:
        print("\n--- SYSTEM MESSAGE: All slots filled! Process complete. ---")
        break

Welcome! I need your destination, date, and class.
Assistant: I have noted N.Y. Can you tell me the date of travel?
Assistant: I have noted December 24. What class would you like to travel in?
Assistant: I have noted First class. Your travel to N.Y is on December 24 in First class.

--- SYSTEM MESSAGE: All slots filled! Process complete. ---
